In [ ]:
# Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.
# SPDX-License-Identifier: MIT-0

# Install missing deps from the `TensorFlow 2.11.0 Python 3.9` instance
!pip install --upgrade pip
!pip install scikit-image

In [ ]:
# REQUIRED: Set pathname which contains the `BBBC005_v1` dataset used for training the model
IMG_PATH = "/root/BBBC005_v1_images/"

# Specify the S3 bucket used for the workshop
BUCKET="YOUR_BUCKET"

# Sample image to process
SAMPLE_IMG="SIMCEPImages_A16_C66_F1_s23_w1.TIF"

# Other test images to try
# SIMCEPImages_A16_C66_F1_s22_w1.TIF
# SIMCEPImages_A16_C66_F1_s22_w2.TIF
# SIMCEPImages_A16_C66_F1_s23_w1.TIF

In [ ]:
import numpy as np
import os
import random
import time
import boto3

import matplotlib.pyplot as plt
%matplotlib inline

from skimage.io import imread, imshow
from skimage.transform import resize

# Don't Show Warning Messages
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Create a new S3 session
session = boto3.Session()
s3 = session.resource('s3')
bucket = s3.Bucket(BUCKET)

print("Uploading ", IMG_PATH + SAMPLE_IMG)

# Upload the specified filename to our S3 bucket, triggering an S3 Event notification for the PUT event
bucket.upload_file(Filename=IMG_PATH + SAMPLE_IMG , Key='input/{}'.format(SAMPLE_IMG))

print("Upload Complete")

print("Polling changes from Lambda invocation ...")

# Confirm the Lambda function is complete, displaying our mask
while True:

    try:
        bucket.download_file('output/{}'.format(SAMPLE_IMG), '/tmp/{}'.format(SAMPLE_IMG))
        print("File processed")
        break
        
    except:
        print("Still processing ...")
        time.sleep(5)

# Read our processed image and display
image = imread('/tmp/{}'.format(SAMPLE_IMG))

plt.xticks([])
plt.yticks([])
plt.title("Cell segmentation mask")
plt.imshow(image)